<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Implementing-your-first-Go-bot" data-toc-modified-id="Implementing-your-first-Go-bot-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Implementing your first Go bot</a></span><ul class="toc-item"><li><span><a href="#gotypes.py" data-toc-modified-id="gotypes.py-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>gotypes.py</a></span></li><li><span><a href="#scoring.py" data-toc-modified-id="scoring.py-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>scoring.py</a></span></li><li><span><a href="#goboard_slow.py" data-toc-modified-id="goboard_slow.py-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>goboard_slow.py</a></span></li><li><span><a href="#helpers.py" data-toc-modified-id="helpers.py-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>helpers.py</a></span></li><li><span><a href="#base.py" data-toc-modified-id="base.py-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>base.py</a></span></li><li><span><a href="#init.py" data-toc-modified-id="init.py-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span><strong>init</strong>.py</a></span></li><li><span><a href="#naive.py" data-toc-modified-id="naive.py-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>naive.py</a></span></li><li><span><a href="#utils.py" data-toc-modified-id="utils.py-3.8"><span class="toc-item-num">3.8&nbsp;&nbsp;</span>utils.py</a></span></li><li><span><a href="#bot_v_bot.py" data-toc-modified-id="bot_v_bot.py-3.9"><span class="toc-item-num">3.9&nbsp;&nbsp;</span>bot_v_bot.py</a></span></li></ul></li></ul></div>

# Deep Learning and the Game of Go

Max Pumperla <br>
Kevin Ferguson <br>

Notes by Michael Ruggiero

## Implementing your first Go bot

I had some trouble following along with the book, so I made this notebook to help myself piece the code together. The book tends to introduce a module, and then comes back to it later. While everything is reasonably labeled, I had a little trouble doubling back so I am going to run through all of the modules part by part.

Here is the basic outline of the code used in chapter 3, with the pycache and .pyc complied work deleted.

.<br>
├── bot_v_bot.py<br>
├── dlgo<br>
│   ├── agent<br>
│   ├── goboard_fast.py<br>
│   ├── goboard_fast.pyc<br>
│   ├── goboard_fast_test.py<br>
│   ├── goboard.py<br>
│   ├── goboard.pyc<br>
│   ├── goboard_slow.py<strong> (Section 3.3)</strong><br>
│   ├── goboard_test.py<br>
│   ├── gotypes.py <strong>(Section 3.1 - 3.2)</strong><br> 
│   ├── __init__.py<br>
│   ├── scoring.py<br>
│   ├── utils.py<br>
│   └── zobrist.py<br>
├── generate_zobrist.py<br>
├── human_v_bot.py<br>
├── requirements.txt<br>
└── setup.py<br>

While this tree is somewhat useful, breaking down the dependencies of each module in this chapter was more helpful piecing everything together in my mind.

In [ ]:
import os
import sys 

path = '/media/data/Documents/Github/Code_and_Presentations/python/GoDeepLearning/code_along'
os.chdir(path + "/dlgo")

#This will set our directory properly
os.getcwd()

### gotypes.py

In [ ]:
%%writefile gotypes.py
#This is a little magic function to write this field into a file

import enum
from collections import namedtuple

#This __all__ is just a list of all of the different classes
#In module 

__all__ = [
    'Player',
    'Point']


class Player(enum.Enum):
    black = 1
    white = 2
    
    
    #A cute little way to switch between players
    @property
    def other(self):
        return Player.black if self == Player.white else Player.white
    
###
#END of Listing 3.1
###

class Point(namedtuple('Point', 'row col')):
    
    #This sets the neighborhood of points the board.
    def neighbors(self):
        
        return [
            Point(self.row - 1, self.col),
            Point(self.row + 1, self.col),
            Point(self.row, self.col - 1),
            Point(self.row, self.col + 1),
        ]

###
#END of Listing 3.2
###
    
    def __deepcopy__(self, memodict={}):
        # These are very immutable.
        return self

### scoring.py

In [ ]:
%%writefile scoring.py

from __future__ import absolute_import
from collections import namedtuple

from dlgo.gotypes import Player, Point


# tag::scoring_territory[]
class Territory:
    def __init__(self, territory_map):  # <1>
        self.num_black_territory = 0
        self.num_white_territory = 0
        self.num_black_stones = 0
        self.num_white_stones = 0
        self.num_dame = 0
        self.dame_points = []
        for point, status in territory_map.items():  # <2>
            if status == Player.black:
                self.num_black_stones += 1
            elif status == Player.white:
                self.num_white_stones += 1
            elif status == 'territory_b':
                self.num_black_territory += 1
            elif status == 'territory_w':
                self.num_white_territory += 1
            elif status == 'dame':
                self.num_dame += 1
                self.dame_points.append(point)

# <1> A `territory_map` splits the board into stones, territory and neutral points (dame).
# <2> Depending on the status of a point, we increment the respective counter.
# end::scoring_territory[]


# tag::scoring_game_result[]
class GameResult(namedtuple('GameResult', 'b w komi')):
    @property
    def winner(self):
        if self.b > self.w + self.komi:
            return Player.black
        return Player.white

    @property
    def winning_margin(self):
        w = self.w + self.komi
        return abs(self.b - w)

    def __str__(self):
        w = self.w + self.komi
        if self.b > w:
            return 'B+%.1f' % (self.b - w,)
        return 'W+%.1f' % (w - self.b,)
# end::scoring_game_result[]


""" evaluate_territory:
Map a board into territory and dame.

Any points that are completely surrounded by a single color are
counted as territory; it makes no attempt to identify even
trivially dead groups.
"""


# tag::scoring_evaluate_territory[]
def evaluate_territory(board):

    status = {}
    for r in range(1, board.num_rows + 1):
        for c in range(1, board.num_cols + 1):
            p = Point(row=r, col=c)
            if p in status:  # <1>
                continue
            stone = board.get(p)
            if stone is not None:  # <2>
                status[p] = board.get(p)
            else:
                group, neighbors = _collect_region(p, board)
                if len(neighbors) == 1:  # <3>
                    neighbor_stone = neighbors.pop()
                    stone_str = 'b' if neighbor_stone == Player.black else 'w'
                    fill_with = 'territory_' + stone_str
                else:
                    fill_with = 'dame'  # <4>
                for pos in group:
                    status[pos] = fill_with
    return Territory(status)

# <1> Skip the point, if you already visited this as part of a different group.
# <2> If the point is a stone, add it as status.
# <3> If a point is completely surrounded by black or white stones, count it as territory.
# <4> Otherwise the point has to be a neutral point, so we add it to dame.
# end::scoring_evaluate_territory[]


""" _collect_region:

Find the contiguous section of a board containing a point. Also
identify all the boundary points.
"""


# tag::scoring_collect_region[]
def _collect_region(start_pos, board, visited=None):

    if visited is None:
        visited = {}
    if start_pos in visited:
        return [], set()
    all_points = [start_pos]
    all_borders = set()
    visited[start_pos] = True
    here = board.get(start_pos)
    deltas = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    for delta_r, delta_c in deltas:
        next_p = Point(row=start_pos.row + delta_r, col=start_pos.col + delta_c)
        if not board.is_on_grid(next_p):
            continue
        neighbor = board.get(next_p)
        if neighbor == here:
            points, borders = _collect_region(next_p, board, visited)
            all_points += points
            all_borders |= borders
        else:
            all_borders.add(neighbor)
    return all_points, all_borders
# end::scoring_collect_region[]


# tag::scoring_compute_game_result[]
def compute_game_result(game_state):
    territory = evaluate_territory(game_state.board)
    return GameResult(
        territory.num_black_territory + territory.num_black_stones,
        territory.num_white_territory + territory.num_white_stones,
        komi=7.5)
# end::scoring_compute_game_result[]

### goboard_slow.py

I found this part very confusing. This is a module that will be revisited over and over again. It is more convenient for me to just define the items in place in one big gulp, rather than jumping back and forth. 

In [ ]:
%%writefile goboard_slow.py

import numpy as np
# tag::imports[]
import copy
from dlgo.gotypes import Player
# end::imports[]
from dlgo.gotypes import Point
from dlgo.scoring import compute_game_result

__all__ = [
    'Board',
    'GameState',
    'Move',
]


class IllegalMoveError(Exception):
    pass

###
#START of Listing 3.4
###

class GoString():  # <1>
    def __init__(self, color, stones, liberties):
        self.color = color
        self.stones = set(stones)
        self.liberties = set(liberties)

    def remove_liberty(self, point):
        self.liberties.remove(point)

    def add_liberty(self, point):
        self.liberties.add(point)

    def merged_with(self, go_string):  # <2>
        assert go_string.color == self.color
        combined_stones = self.stones | go_string.stones
        return GoString(
            self.color,
            combined_stones,
            (self.liberties | go_string.liberties) - combined_stones)

    @property
    def num_liberties(self):
        return len(self.liberties)

    def __eq__(self, other):
        return isinstance(other, GoString) and \
            self.color == other.color and \
            self.stones == other.stones and \
            self.liberties == other.liberties
# <1> Go strings are stones that are linked by a chain of connected stones of the same color.
# <2> Return a new Go string containing all stones in both strings.

###
#END of Listing 3.4
###


###
#START of Listing 3.5
###

class Board():  # <1>
    def __init__(self, num_rows, num_cols):
        self.num_rows = num_rows
        self.num_cols = num_cols
        self._grid = {}

# <1> A board is initialized as empty grid with the specified number of rows and columns.

###
#END of Listing 3.5
###

###
#Start of Listing 3.6
###


# tag::board_place_0[]
    def place_stone(self, player, point):
        assert self.is_on_grid(point)
        assert self._grid.get(point) is None
        adjacent_same_color = []
        adjacent_opposite_color = []
        liberties = []
        for neighbor in point.neighbors():  # <1>
            if not self.is_on_grid(neighbor):
                continue
            neighbor_string = self._grid.get(neighbor)
            if neighbor_string is None:
                liberties.append(neighbor)
            elif neighbor_string.color == player:
                if neighbor_string not in adjacent_same_color:
                    adjacent_same_color.append(neighbor_string)
            else:
                if neighbor_string not in adjacent_opposite_color:
                    adjacent_opposite_color.append(neighbor_string)
        new_string = GoString(player, [point], liberties)
# <1> First, we examine direct neighbors of this point.

###
#END of Listing 3.6
###

###
#START of Listing 3.8
###

# end::board_place_0[]
# tag::board_place_1[]
        for same_color_string in adjacent_same_color:  # <1>
            new_string = new_string.merged_with(same_color_string)
        for new_string_point in new_string.stones:
            self._grid[new_string_point] = new_string
        for other_color_string in adjacent_opposite_color:  # <2>
            other_color_string.remove_liberty(point)
        for other_color_string in adjacent_opposite_color:  # <3>
            if other_color_string.num_liberties == 0:
                self._remove_string(other_color_string)
# <1> Merge any adjacent strings of the same color.
# <2> Reduce liberties of any adjacent strings of the opposite color.
# <3> If any opposite color strings now have zero liberties, remove them.
# end::board_place_1[]

###
#END of Listing 3.8
###

###
#START of Listing 3.9
###

# tag::board_remove[]
    def _remove_string(self, string):
        for point in string.stones:
            for neighbor in point.neighbors():  # <1>
                neighbor_string = self._grid.get(neighbor)
                if neighbor_string is None:
                    continue
                if neighbor_string is not string:
                    neighbor_string.add_liberty(point)
            self._grid[point] = None
# <1> Removing a string can create liberties for other strings.
# end::board_remove[]

###
#End of Listing 3.9
###


# tag::board_utils[]

###
#Start of Listing 3.7
###

    def is_on_grid(self, point):
        return 1 <= point.row <= self.num_rows and \
            1 <= point.col <= self.num_cols

    def get(self, point):  # <1>
        string = self._grid.get(point)
        if string is None:
            return None
        return string.color

    def get_go_string(self, point):  # <2>
        string = self._grid.get(point)
        if string is None:
            return None
        return string
# <1> Returns the content of a point on the board:  a Player if there is a stone on that point or else None.
# <2> Returns the entire string of stones at a point: a GoString if there is a stone on that point or else None.
# end::board_utils[]

###
#END of Listing 3.7
###


    def __eq__(self, other):
        return isinstance(other, Board) and \
            self.num_rows == other.num_rows and \
            self.num_cols == other.num_cols and \
            self._grid == other._grid



###
#START OF LISTING 3.3
###

class Move():  # <1>
    def __init__(self, point=None, is_pass=False, is_resign=False):
        assert (point is not None) ^ is_pass ^ is_resign
        self.point = point
        self.is_play = (self.point is not None)
        self.is_pass = is_pass
        self.is_resign = is_resign

    @classmethod
    def play(cls, point):  # <2>
        return Move(point=point)

    @classmethod
    def pass_turn(cls):  # <3>
        return Move(is_pass=True)

    @classmethod
    def resign(cls):  # <4>
        return Move(is_resign=True)
# <1> Any action a player can play on a turn, either is_play, is_pass or is_resign will be set.
# <2> This move places a stone on the board.
# <3> This move passes.
# <4> This move resigns the current game

###
#END of Listing 3.3
###

###
#START of Listing 3.10
###


# tag::game_state[]
class GameState():
    def __init__(self, board, next_player, previous, move):
        self.board = board
        self.next_player = next_player
        self.previous_state = previous
        self.last_move = move

    def apply_move(self, move):  # <1>
        if move.is_play:
            next_board = copy.deepcopy(self.board)
            next_board.place_stone(self.next_player, move.point)
        else:
            next_board = self.board
        return GameState(next_board, self.next_player.other, self, move)

    @classmethod
    def new_game(cls, board_size):
        if isinstance(board_size, int):
            board_size = (board_size, board_size)
        board = Board(*board_size)
        return GameState(board, Player.black, None, None)
# <1> Return the new GameState after applying the move.
# end::game_state[]

###
#END of Listing 3.10
###

###
#START of Listing 3.12
###

# tag::self_capture[]
    def is_move_self_capture(self, player, move):
        if not move.is_play:
            return False
        next_board = copy.deepcopy(self.board)
        next_board.place_stone(player, move.point)
        new_string = next_board.get_go_string(move.point)
        return new_string.num_liberties == 0
# end::self_capture[]

###
#END of Listing 3.12
###

###
#START of Listing 3.13
###

# tag::is_ko[]
    @property
    def situation(self):
        return (self.next_player, self.board)

    def does_move_violate_ko(self, player, move):
        if not move.is_play:
            return False
        next_board = copy.deepcopy(self.board)
        next_board.place_stone(player, move.point)
        next_situation = (player.other, next_board)
        past_state = self.previous_state
        while past_state is not None:
            if past_state.situation == next_situation:
                return True
            past_state = past_state.previous_state
        return False
# end::is_ko[]

###
#END of Listing 3.13
###

###
#START of Listing 3.14
###

# tag::is_valid_move[]
    def is_valid_move(self, move):
        if self.is_over():
            return False
        if move.is_pass or move.is_resign:
            return True
        return (
            self.board.get(move.point) is None and
            not self.is_move_self_capture(self.next_player, move) and
            not self.does_move_violate_ko(self.next_player, move))
# end::is_valid_move[]

###
#END of Listing 3.14
###


# tag::is_over[]

###
#START of Listing 3.11
###

    def is_over(self):
        if self.last_move is None:
            return False
        if self.last_move.is_resign:
            return True
        second_last_move = self.previous_state.last_move
        if second_last_move is None:
            return False
        return self.last_move.is_pass and second_last_move.is_pass
    
###
#END of Listing 3.11
###

# end::is_over[]

    def legal_moves(self):
        moves = []
        for row in range(1, self.board.num_rows + 1):
            for col in range(1, self.board.num_cols + 1):
                move = Move.play(Point(row, col))
                if self.is_valid_move(move):
                    moves.append(move)
        # These two moves are always legal.
        moves.append(Move.pass_turn())
        moves.append(Move.resign())

        return moves

    def winner(self):
        if not self.is_over():
            return None
        if self.last_move.is_resign:
            return self.next_player
        game_result = compute_game_result(self)
        return game_result.winner

### helpers.py

In [ ]:
os.chdir(path + "/dlgo/agent")
os.getcwd()

In [ ]:
%%writefile helpers.py

###
#Start Listing 3.15
###


# tag::helpersimport[]
from dlgo.gotypes import Point
# end::helpersimport[]

__all__ = [
    'is_point_an_eye',
]


# tag::eye[]
def is_point_an_eye(board, point, color):
    if board.get(point) is not None:  # <1>
        return False
    for neighbor in point.neighbors():  # <2>
        if board.is_on_grid(neighbor):
            neighbor_color = board.get(neighbor)
            if neighbor_color != color:
                return False

    friendly_corners = 0  # <3>
    off_board_corners = 0
    corners = [
        Point(point.row - 1, point.col - 1),
        Point(point.row - 1, point.col + 1),
        Point(point.row + 1, point.col - 1),
        Point(point.row + 1, point.col + 1),
    ]
    for corner in corners:
        if board.is_on_grid(corner):
            corner_color = board.get(corner)
            if corner_color == color:
                friendly_corners += 1
        else:
            off_board_corners += 1
    if off_board_corners > 0:
        return off_board_corners + friendly_corners == 4  # <4>
    return friendly_corners >= 3  # <5>

# <1> An eye is an empty point.
# <2> All adjacent points must contain friendly stones.
# <3> We must control 3 out of 4 corners if the point is in the middle of the board; on the edge we must control all corners.
# <4> Point is on the edge or corner.
# <5> Point is in the middle.
# end::eye[]

###
#END of Listing 3.15
###


### base.py

In [ ]:
%%writefile base.py

__all__ = [
    'Agent',
]

###
#START of Listing 3.16
###
# tag::agent[]
class Agent:
    def __init__(self):
        pass

    def select_move(self, game_state):
        raise NotImplementedError()
# end::agent[]

###
#END of Listing 3.16
###

    def diagnostics(self):
        return {}


### __init__.py

In [ ]:
%%writefile __init__.py


from .base import *
#from .naive import *
#from .naive_fast import *


### naive.py

In [ ]:
%%writefile naive.py

###
#START of Listing 3.17
###

# tag::randombotimports[]
import random
from dlgo.agent.base import Agent
from dlgo.agent.helpers import is_point_an_eye
from dlgo.goboard_slow import Move
from dlgo.gotypes import Point
# end::randombotimports[]


__all__ = ['RandomBot']


# tag::random_bot[]
class RandomBot(Agent):
    def select_move(self, game_state):
        """Choose a random valid move that preserves our own eyes."""
        candidates = []
        for r in range(1, game_state.board.num_rows + 1):
            for c in range(1, game_state.board.num_cols + 1):
                candidate = Point(row=r, col=c)
                if game_state.is_valid_move(Move.play(candidate)) and \
                        not is_point_an_eye(game_state.board,
                                            candidate,
                                            game_state.next_player):
                    candidates.append(candidate)
        if not candidates:
            return Move.pass_turn()
        return Move.play(random.choice(candidates))
# end::random_bot[]

###
#END of Listing 3.17
###


### utils.py

In [ ]:
os.chdir(path + "/dlgo")
os.getcwd()

In [ ]:
%%writefile utils.py

###
#START of Listing 3.18
###

import numpy as np
# tag::print_utils[]
from dlgo import gotypes

COLS = 'ABCDEFGHJKLMNOPQRST'
STONE_TO_CHAR = {
    None: ' . ',
    gotypes.Player.black: ' x ',
    gotypes.Player.white: ' o ',
}


def print_move(player, move):
    if move.is_pass:
        move_str = 'passes'
    elif move.is_resign:
        move_str = 'resigns'
    else:
        move_str = '%s%d' % (COLS[move.point.col - 1], move.point.row)
    print('%s %s' % (player, move_str))


def print_board(board):
    for row in range(board.num_rows, 0, -1):
        bump = " " if row <= 9 else ""
        line = []
        for col in range(1, board.num_cols + 1):
            stone = board.get(gotypes.Point(row=row, col=col))
            line.append(STONE_TO_CHAR[stone])
        print('%s%d %s' % (bump, row, ''.join(line)))
    print('    ' + '  '.join(COLS[:board.num_cols]))
# end::print_utils[]

###
#END of Listing 3.18
###

# tag::human_coordinates[]
def point_from_coords(coords):
    col = COLS.index(coords[0]) + 1
    row = int(coords[1:])
    return gotypes.Point(row=row, col=col)
# end::human_coordinates[]


def coords_from_point(point):
    return '%s%d' % (
        COLS[point.col - 1],
        point.row
    )

# NOTE: MoveAge is only used in chapter 13, and doesn't make it to the main text.
# This feature will only be implemented in goboard_fast.py so as not to confuse
# readers in early chapters.
class MoveAge():
    def __init__(self, board):
        self.move_ages = - np.ones((board.num_rows, board.num_cols))

    def get(self, row, col):
        return self.move_ages[row, col]

    def reset_age(self, point):
        self.move_ages[point.row - 1, point.col - 1] = -1

    def add(self, point):
        self.move_ages[point.row - 1, point.col - 1] = 0

    def increment_all(self):
        self.move_ages[self.move_ages > -1] += 1

### bot_v_bot.py

In [ ]:
os.chdir(path)
os.getcwd()

In [ ]:
%%writefile bot_v_bot.py

###
#START of Listing 3.19
###

from __future__ import print_function
# tag::bot_vs_bot[]
from dlgo import agent
from dlgo import goboard_slow as goboard
from dlgo import gotypes
from dlgo.utils import print_board, print_move
import time


def main():
    board_size = 9
    game = goboard.GameState.new_game(board_size)
    bots = {
        gotypes.Player.black: agent.naive.RandomBot(),
        gotypes.Player.white: agent.naive.RandomBot(),
    }
    while not game.is_over():
        #Changed the speed to make it quicker
        time.sleep(0.1)  # <1>

        print(chr(27) + "[2J")  # <2>
        print_board(game.board)
        bot_move = bots[game.next_player].select_move(game)
        
        #Commented out the print_move since it made an unpleasant jiggering of printout 
        #print_move(game.next_player, bot_move)
        game = game.apply_move(bot_move)


if __name__ == '__main__':
    main()

# <1> We set a sleep timer to 0.3 seconds so that bot moves aren't printed too fast to observe
# <2> Before each move we clear the screen. This way the board is always printed to the same position on the command line.
# end::bot_vs_bot[]

###
#END of Listing 3.19
###

In [ ]:
import sys
sys.path

In [ ]:
import sys
sys.path.append(path + "/dlgo")

import bot_v_bot
bot_v_bot.main()

In [ ]:
import bot_v_bot
import dlgo.agent